In [4]:
import pandas as pd
import os
working_dir = './user_logs_copy'

users=[]
for root, dirs, _ in os.walk(working_dir):
    for d in dirs:
        if d.startswith('user-'):
            users.append(d)
users=sorted(users, key=lambda item: int(item.partition('-')[2]))
    
n_devices=[]
user_call_logs=[]
user_text_logs=[]
user_contact_logs=[]
for user in users:
    n_devices.append(len(next(os.walk(working_dir+'/'+user))[1]))
    
    for u, device, files in os.walk(working_dir+'/'+user):
        if not u.endswith(user): # solution for device folder "u" repetition
            for filename in files:
                if filename.endswith('call_log.txt'):
                    user_call_logs.append(os.path.join(u, filename)) 
                if filename.endswith('sms_log.txt'):
                    user_text_logs.append(os.path.join(u, filename))
                if filename.endswith('list.txt'):
                    user_contact_logs.append(os.path.join(u, filename))

    #user_calls = [pd.read_table(user_call_log) for user_call_log in user_call_logs]
    #user_texts = [pd.read_table(user_text_log) for user_text_log in user_text_logs]
    #user_contacts = [pd.read_table(user_contact_list) for user_contact_list in user_contact_lists]
    
user_df = pd.DataFrame({'n_devices':n_devices})
#print(user_df)

In [6]:
#df = users_df + add results and IDs columns + n_devices
print(master_df)

NameError: name 'master_df' is not defined